In [1]:
#Exercise1 
import numpy as np
from sklearn.model_selection import KFold

X = np.array(np.arange(1, 21).reshape(10, -1))
y = np.array(np.arange(1, 11))
kf = KFold(n_splits=5)

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    print(f"Fold: {fold}")
    print(f"TRAIN: {train_index} TEST: {test_index}\n")
    



Fold: 1
TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]

Fold: 2
TRAIN: [0 1 4 5 6 7 8 9] TEST: [2 3]

Fold: 3
TRAIN: [0 1 2 3 6 7 8 9] TEST: [4 5]

Fold: 4
TRAIN: [0 1 2 3 4 5 8 9] TEST: [6 7]

Fold: 5
TRAIN: [0 1 2 3 4 5 6 7] TEST: [8 9]



In [2]:
#Exercise2
# imports
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# data
housing = fetch_california_housing()
X, y = housing['data'], housing['target']

# split data train test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=43)

# pipeline
pipeline = [('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
            ('lr', LinearRegression())]
pipe = Pipeline(pipeline)

from sklearn.model_selection import cross_validate

# Cross-validate with 10 folds
cv_results = cross_validate(pipe, X_train, y_train, cv=10, scoring='r2')

scores = cv_results['test_score']
print("Scores on validation sets:")
print(scores)

mean_score = scores.mean()
print("\nMean of scores on validation sets:")
print(mean_score)

std_score = scores.std()
print("\nStandard deviation of scores on validation sets:")
print(std_score)

Scores on validation sets:
[0.62433594 0.61648956 0.62486602 0.59891024 0.59284295 0.61307055
 0.54630341 0.60742976 0.60014575 0.59574508]

Mean of scores on validation sets:
0.6020139252674299

Standard deviation of scores on validation sets:
0.02149838227734666


In [ ]:
#Exercise 3
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# data
housing = fetch_california_housing()
X, y = housing['data'], housing['target']
# split data train test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=43)

param_grid = {
    'max_depth': [1, 5, 10, 15, 20],  # Tree depth: deeper trees can overfit
    'n_estimators': [10, 50, 100]     # Number of trees: more trees reduce variance but increase time
}

rf = RandomForestRegressor(random_state=43)  

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Negative MSE (since GridSearch maximizes)
    n_jobs=-1  # Use all CPUs
)

grid_search.fit(X_train, y_train)



In [ ]:
best_estimator = grid_search.best_estimator_
print("Best parameters:")
print(grid_search.best_params_)

print("\nBest score on validation sets (neg MSE):")
print(grid_search.best_score_)

print("\nCV results:")
print(grid_search.cv_results_)

In [ ]:
test_score = best_estimator.score(X_test, y_test)
print("\nScore on test set (R²):")
print(test_score)

In [ ]:
#Exercise4
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import validation_curve, learning_curve

X, y = make_classification(n_samples=100000,
                        n_features=30,
                        n_informative=10,
                        flip_y=0.2,
                        random_state=42)

param_range = np.arange(1, 21, 2)  

train_scores, valid_scores = validation_curve(
    RandomForestClassifier(random_state=42),
    X, y,
    param_name="max_depth",
    param_range=param_range,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

# Calculate means and stds
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
valid_mean = np.mean(valid_scores, axis=1)
valid_std = np.std(valid_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(param_range, train_mean, label="Training score", color="blue")
plt.plot(param_range, valid_mean, label="Validation score", color="orange")
plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color="blue")
plt.fill_between(param_range, valid_mean - valid_std, valid_mean + valid_std, alpha=0.1, color="orange")
plt.title("Validation curve : max_depth")
plt.xlabel("max_depth")
plt.ylabel("Score : ROC AUC")
plt.legend(loc="best")

def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, validation_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy'
    )
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    validation_scores_mean = np.mean(validation_scores, axis=1)
    validation_scores_std = np.std(validation_scores, axis=1)
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, validation_scores_mean, 'o-', color="g", label="Cross-validation score")
    plt.legend(loc="best")
    plt.show()

plot_learning_curve(RandomForestClassifier(max_depth=12, random_state=42), 
                    "Learning Curve (Random Forest)", X, y, cv=10, n_jobs=-1)

